Data Wrangling Notebook for SY Data
<br />
Neeka Sewnath
<br />
nsewnath@ufl.edu

In [1]:
import pandas as pd
from pandas import Series
import numpy as np
import re
import uuid 

Silencing warnings that are unnecessary

In [2]:
try:
    import warnings
    warnings.filterwarnings('ignore')
except:
    pass

Import SY Data

In [3]:
sy_data = pd.read_csv("./../Original_Data/futres_sy_mod.csv")

Rename Columns

In [4]:
sy_data = sy_data.rename(columns={'ID': 'individualID',
                                  'Taxa':'scientificName',
                                  'Measurements (NAME;NAME;NAME...) divided by ;':'measurementType',
                                  'Values (mm;mm;mm...) divided by ;':'measurementValue',
                                  'Gender': 'sex',
                                  'Age appearance':'lifeStage'})

Clean up lifeStage column

In [5]:
# Create Filters
adult = sy_data['lifeStage']=="ADL"
juvenile = sy_data['lifeStage']=="JUV"

# Assign correct terms using filters
sy_data['lifeStage'][adult] = "adult"
sy_data['lifeStage'][juvenile] = "juvenile"

Clean up sex column

In [6]:
male = sy_data['sex']=="M"
sy_data["sex"][male==True]="male"
sy_data['sex'][male==False]=""

Check is length of measurementType and measurementValue are the same 

In [7]:
#def matching_size(row):
#    """function to match size of measurement type and value"""
#    mtype = row["measurementType"].split(';')
#    mval = row["measurementValue"].split(';')
#    if len(mtype) != len(mval):
#        print ("Error: Size Mismatch")
#        print(row.name)
        
#sy_data.apply(matching_size,  axis =1)

JSON specific columns and add to dynamicProperties

In [18]:
# Specify columns for dynamic properties
json_columns = sy_data.iloc[:,19:45]

# Create dynamicProperties
sy_data = sy_data.assign(dynamicProperties="")

for i in json_columns.index:
    sy_data["dynamicProperties"][i] = json_columns.loc[i].to_json()

Subsetting by avaliable terms